In [28]:
import numpy as np #import numpy library
import pandas as pd #import pandas library
import matplotlib.pyplot as plt #import matplot library
%matplotlib inline
import nltk.classify.util
from astropy.table import Table, Column

import string

from sklearn import svm
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [29]:
data = pd.read_csv('Amazon.csv') #attach Amazon data to a var called data
# can also include a http addy in there.... 
print(data.shape) #print data rows and columns
data.head(5) #limit data to 5 rows including a header row 

(455000, 13)


,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,helpScore,helpful
0,138806,138807,B000E63LME,A1CQGW1AOD0LF2,"Alena K. ""Alena""",1,2,2,1294185600,Not as pictured.,I was looking forward to try cranberry apple f...,0.5,False
1,469680,469681,B004ZIH4KM,A37S7U1OX2MCWI,Becky Cole,0,0,5,1349740800,seeds,"TY for everything. The seeds arrived quickly,...",NaN,False
2,238202,238203,B003ZXE9QA,A2OM6G73E64EQ9,jeff,0,0,5,1329264000,I'm addicted!,I've finally found the best cereal in the worl...,NaN,False
3,485307,485308,B001RVFERK,A25W349EE97NBK,Tangent4,1,1,4,1248307200,I wanted to love these...,I originally bought these chips because I'd he...,1.0,False
4,375283,375284,B000OQZNTS,A3CPPW0HUC07YS,Amy Nicolai,0,0,5,1333238400,Excellent chamomile tea,"Really excellent tea, flowers are visible in t...",NaN,False


In [30]:
# data.tail()


In [31]:
#smData = data[0:1000] #make a variabe for data subset limited to first 1000 
#smData.shape #show how many rows and columns we have
#print(smData.shape) #print data rows and columns
#smData.head(5) #limit data to 5 rows including a header row 

In [32]:
# vectorize Bag of Words from review text; as sparse matrix
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(min_df=1, ngram_range=(3,3))
X = tv.fit_transform(data.Text)

In [33]:
# # vectorize Bag of Words from review text; as sparse matrix
# from sklearn.feature_extraction.text import HashingVectorizer
# hv = HashingVectorizer(n_features=2 ** 17, non_negative=True)
# X = hv.transform(data.Text)
# X.shape
# (455000, 131072)

# CRASHES!!!
# # vectorize Bag of Words from review text; as sparse matrix
# from sklearn.feature_extraction.text import TfidfVectorizer
# tv = TfidfVectorizer(min_df=1, ngram_range=(2,17))
# X = tv.fit_transform(data.Text)


In [34]:
#convert summary NaNs into usable format -- thank you Andy!
data.Summary = data.Summary.fillna('none')
# data.ProfileName = data.ProfileName.fillna('none')
# data.Score = data.Score.fillna('none')

XSummary = hv.transform(data.Summary)
XSummary.shape

(455000, 131072)

In [35]:
#convert unix timestamp to datetime -- thanks Tyler!
data['DateTime'] = pd.to_datetime(data['Time'], unit='s')

In [36]:
#features extraction function -- thanks elena
def featuresExtract(data):
    '''Takes as an argument the original dataframe
       Extracts number of unique users and products
       as well as number of repeat users and product to user ratio
       Returns Pandas dataframe with new features
    '''

    # how many reviews for the same product in test set - products with more reviews may have more helpful comments as they are more popular
    nReviewsProducts = data.groupby(['ProductId'])['Id'].transform('count')

    # ratio of n reviews to product
    nReviewsRatio = 1 / data.groupby(['ProductId'])['UserId'].transform('count')

    # n repeat users - hypothesis is that people who comment more may provide more helpful reviews
    nRepeatUsers = data.groupby(['UserId'])['ProductId'].transform('count')

    # n unique users
    nUniqueUsers = data.groupby(['ProductId'])['UserId'].transform('count')
    
    # first review
    first = data['Time'].rank(ascending=1)
    
    #data and time

    # all these Times are midnight of a day

    # find day of week (0-6, Mon-Sun)
    day = data['DateTime'].dt.weekday
    month = data['DateTime'].dt.month
    year = data['DateTime'].dt.year


    names = np.array([ 'nReviewsProducts', 'nReviewsRatio', 'nRepeatUsers', 'nUniqueUsers', 'first_review', 'day', 'month', 'year' ])
    result = np.array([ nReviewsProducts, nReviewsRatio, nRepeatUsers, nUniqueUsers, first, day, month, year ]).T
    myFeatures = pd.DataFrame( result, columns = names )


    return myFeatures

In [37]:
user_analysis = featuresExtract(data)

In [38]:
#features extraction function -- thanks elena
def textProcessor(reviewText):
    '''Takes as an argument the text of the reviews
       Performs a series of transformations
       Returns Pandas dataframe with ten new features
    '''

    # count number of words in review
    nWords = [len(w.split()) for w in reviewText]

    # count number of characters in review
    nChar = [len(c) for c in reviewText]

    # words to characters ratio
    wordCharRatio = [ w/c for w,c in zip( map(float, nWords), map(float, nChar) ) ]

    # number of uppercase words
    nUpper =  [ np.sum( [w.isupper() for w in word.split()] ) for word in reviewText ]

    # uppercase to word ratio
    upperRatio = np.array(nUpper) / np.array(nWords, dtype=float)

    # longest word
    exclude = string.punctuation
    words = [ word.split() for word in reviewText ]
    noPunct = [ [''.join(k for k in c if k not in exclude) for c in w ] for w in words]
    longestWord = [ np.max( [len(w) for w in word] ) for word in noPunct ]

    # average word length
    avgWordLen = [ np.mean( [len(w) for w in word.split()] ) for word in reviewText ]

    # punctuation count
    punctCount = [ np.sum([c.count(p) for p in string.punctuation]) for c in reviewText ]

    # punctuation ratio to character ration
    punctRatio = [ p/c for p,c in zip( map(float, punctCount), map(float, nChar) ) ]

    # exclamation point count
    exclamationPoint = [e.count("!") for e in reviewText]

    names = np.array([ 'nWords', 'nChar', 'wordCharRatio', 'nUpper', 'upperRatio', 'longestWord', 'avgWordLen', 'exclamationPoint', 'punctCount', 'punctRatio' ])
    result = np.array([ nWords, nChar, wordCharRatio, nUpper, upperRatio, longestWord, avgWordLen, exclamationPoint, punctCount, punctRatio ]).T

    myFeatures = pd.DataFrame( result, columns = names )
    # print(myFeatures)
    # print(myFeatures.shape)
    # display(myFeatures)

    return myFeatures


In [39]:
# print(data.Summary)
# print(data.Text)

In [40]:
text_data = textProcessor(data.Text)

In [41]:
summary_data = textProcessor(data.Summary)

In [42]:
X_counts = np.concatenate((text_data, summary_data, user_analysis), axis = 1)

print(X_counts)

[[  3.20000000e+01   2.06000000e+02   1.55339806e-01 ...,   2.00000000e+00
    1.00000000e+00   2.01100000e+03]
 [  2.60000000e+01   1.25000000e+02   2.08000000e-01 ...,   1.00000000e+00
    1.00000000e+01   2.01200000e+03]
 [  1.24000000e+02   6.78000000e+02   1.82890855e-01 ...,   2.00000000e+00
    2.00000000e+00   2.01200000e+03]
 ..., 
 [  4.40000000e+01   2.83000000e+02   1.55477032e-01 ...,   0.00000000e+00
    8.00000000e+00   2.01100000e+03]
 [  4.60000000e+01   2.66000000e+02   1.72932331e-01 ...,   1.00000000e+00
    6.00000000e+00   2.01000000e+03]
 [  5.30000000e+01   2.75000000e+02   1.92727273e-01 ...,   3.00000000e+00
    5.00000000e+00   2.01200000e+03]]


In [43]:
# features from Amazon.csv to add to feature set
# data['reviewLen'] = data['Text'].str.len() #captures the amount of strings present in the review (i.e. review length)
# Features pulled from review txt
data['hasEP'] = data.Text.str.contains('!') #captures the presences of an exclaimation point
data['hasSemi'] = data.Text.str.contains(';') #captures the presences of a ;
data['hasColon'] = data.Text.str.contains(':') #captures the presences of a :
data['hasAt'] = data.Text.str.contains('@') #captures the presences of a @
data['hasDolla'] = data.Text.str.contains('$') #captures the presences of a $
data['hasNumb'] = data.Text.str.contains('#') #captures the presences of a #
data['hasAnd'] = data.Text.str.contains('&') #captures the presences of a &
data['hasQuestion'] = data.Text.str.contains('\?') #captures the presences of a ?
data['hasAsterick'] = data.Text.str.contains('\*') #captures the presences of a *
data['hasFparenthesis'] = data.Text.str.contains('\(') #captures the presences of a (
data['hasBparenthesis'] = data.Text.str.contains('\)') #captures the presences of a )
data['epCount'] = data.Text.str.count('!')
data['semiCount'] = data.Text.str.count(':')
data['atCount'] = data.Text.str.count('@')
data['dollaCount'] = data.Text.str.count('$')
data['numbCount'] = data.Text.str.count('#')
data['andCount'] = data.Text.str.count('&')
data['questionCount'] = data.Text.str.count('\?')
data['asterickCount'] = data.Text.str.count('\*')
data['fParenthesisCount'] = data.Text.str.count('\(')
data['bParenthesisCount'] = data.Text.str.count('\)')

# # add punctuation count and ratio of punctuation to words -- thank you jon!
# data['punctuationCount'] = ddata.Text.str.count('[.,!;:()/\?-@#$&]') #captures total common punctuation count
# data['punctuationToWords'] = data['punctuationCount'] / data['reviewLen'] #captures ratio of punctuation to words

# # find repeating reviews
data['repeatTxt'] = data.duplicated('Text')

# # add vars from profile name 
data['nameNumb'] = data['ProfileName'].str.contains('#') #captures profile name with a number sign
data['nameQuestion'] = data['ProfileName'].str.contains('\?')#captures profile name with a question mark
data['nameAsterick'] = data['ProfileName'].str.contains('\*') #captures profile name with an asterick
data['nameAnd'] = data['ProfileName'].str.contains('&') #captures the presences of a &
data['nameSemi'] = data['ProfileName'].str.contains(';') #captures the presences of a ;
data['nameColon'] = data['ProfileName'].str.contains(':') #captures the presences of a :
data['nameAt'] = data['ProfileName'].str.contains('@') #captures the presences of a @
data['nameDolla'] = data['ProfileName'].str.contains('$') #captures the presences of a $

# # finding repeating profile names 
# data['repeatProfile'] = data.duplicated('ProfileName')

# # Features pulled from summary txt
# data['summaryLen'] = data['Summary'].str.len() #captures the amount of strings present in the review (i.e. review length)
data['summaryEP'] = data['Summary'].str.contains('!') #captures the presences of an exclaimation point
data['summarySemi'] = data['Summary'].str.contains(';') #captures the presences of a ;
data['summaryColon'] = data['Summary'].str.contains(':') #captures the presences of a :
data['summaryAt'] = data['Summary'].str.contains('@') #captures the presences of a @
data['summaryDolla'] = data['Summary'].str.contains('$') #captures the presences of a $
data['summaryNumb'] = data['Summary'].str.contains('#') #captures the presences of a #
data['summaryAnd'] = data['Summary'].str.contains('&') #captures the presences of a &
data['summaryQuestion'] = data['Summary'].str.contains('\?') #captures the presences of a ?
data['summaryAsterick'] = data['Summary'].str.contains('\*') #captures the presences of a *
data['summaryFparenthesis'] = data['Summary'].str.contains('\(') #captures the presences of a (
data['summaryBparenthesis'] = data['Summary'].str.contains('\)') #captures the presences of a )
data['summaryEpCount'] = data['Summary'].str.count('!')
data['summarySemiCount'] = data['Summary'].str.count(':')
data['summaryAtCount'] = data['Summary'].str.count('@')
data['summaryDollaCount'] = data['Summary'].str.count('$')
data['summaryNumbCount'] = data['Summary'].str.count('#')
data['summaryAndCount'] = data['Summary'].str.count('&')
data['summaryQuestionCount'] = data['Summary'].str.count('\?')
data['summaryAsterickCount'] = data['Summary'].str.count('\*')
data['summaryFParenthesisCount'] = data['Summary'].str.count('\(')
data['summaryBParenthesisCount'] = data['Summary'].str.count('\)')

# # add punctuation count and ratio of punctuation to words -- thank you jon!
# data['summaryPunctuationCount'] = data['Summary'].str.count('[.,!;:()/\?-@#$&]') #captures total common punctuation count
# data['summaryPunctuationToWords'] = data['punctuationCount'] / data['summaryLen'] #captures ratio of punctuation to words

# # find repeating reviews
data['summaryRepeatTxt'] = data.duplicated('Summary')


In [44]:
# Taking values and making vectors
# XScore = data.score.values.reshape(data.shape[0], 1)
# XreviewLen = data.iloc[:, 13].values.reshape(data.shape[0], 1)
XhasEP = data.hasEP.values.reshape(data.shape[0], 1)
XhasSemi = data.hasSemi.values.reshape(data.shape[0], 1)
XhasColon = data.hasColon.values.reshape(data.shape[0], 1)
hasAt = data.hasAt.values.reshape(data.shape[0], 1)
hasDolla = data.hasDolla.values.reshape(data.shape[0], 1)
hasNumb = data.hasNumb.values.reshape(data.shape[0], 1)
hasAnd = data.hasAnd.values.reshape(data.shape[0], 1)
hasQuestion = data.hasQuestion.values.reshape(data.shape[0], 1)
hasAsterick = data.hasAsterick.values.reshape(data.shape[0], 1)
hasFparenthesis = data.hasFparenthesis.values.reshape(data.shape[0], 1)
hasBparenthesis = data.hasBparenthesis.values.reshape(data.shape[0], 1)
epCount = data.epCount.values.reshape(data.shape[0], 1)
semiCount = data.semiCount.values.reshape(data.shape[0], 1)
atCount = data.atCount.values.reshape(data.shape[0], 1)
dollaCount = data.dollaCount.values.reshape(data.shape[0], 1)
numbCount = data.numbCount.values.reshape(data.shape[0], 1)
andCount = data.andCount.values.reshape(data.shape[0], 1)
questionCount = data.questionCount.values.reshape(data.shape[0], 1)
asterickCount = data.asterickCount.values.reshape(data.shape[0], 1)
fParenthesisCount = data.fParenthesisCount.values.reshape(data.shape[0], 1)
bParenthesisCount = data.bParenthesisCount.values.reshape(data.shape[0], 1)
# punctuationCount = data.iloc[:, 35].values.reshape(data.shape[0], 1)
# punctuationToWords = data.iloc[:, 36].values.reshape(data.shape[0], 1)
repeatTxt = data.repeatTxt.values.reshape(data.shape[0], 1)
nameNumb = data.nameNumb.values.reshape(data.shape[0], 1)
nameQuestion = data.nameQuestion.values.reshape(data.shape[0], 1)
nameAsterick = data.nameAsterick.values.reshape(data.shape[0], 1)
nameAnd = data.nameAnd.values.reshape(data.shape[0], 1)
nameSemi = data.nameSemi.values.reshape(data.shape[0], 1)
nameColon = data.nameColon.values.reshape(data.shape[0], 1)
nameAt = data.nameAt.values.reshape(data.shape[0], 1)
nameDolla = data.nameDolla.values.reshape(data.shape[0], 1)
# repeatProfile = data.repeatProfile.values.reshape(data.shape[0], 1)
# summaryLen = data.iloc[:, 47].values.reshape(data.shape[0], 1)
summaryEP = data.summaryEP.values.reshape(data.shape[0], 1)
summarySemi = data.summarySemi.values.reshape(data.shape[0], 1)
summaryColon = data.summaryColon.values.reshape(data.shape[0], 1)
summaryAt = data.summaryAt.values.reshape(data.shape[0], 1)
summaryDolla = data.summaryDolla.values.reshape(data.shape[0], 1)
summaryNumb = data.summaryNumb.values.reshape(data.shape[0], 1)
summaryAnd = data.summaryAnd.values.reshape(data.shape[0], 1)
summaryQuestion = data.summaryQuestion.values.reshape(data.shape[0], 1)
summaryAsterick = data.summaryAsterick.values.reshape(data.shape[0], 1)
summaryFparenthesis = data.summaryFparenthesis.values.reshape(data.shape[0], 1)
summaryBparenthesis = data.summaryBparenthesis.values.reshape(data.shape[0], 1)
summaryEpCount = data.summaryEpCount.values.reshape(data.shape[0], 1)
summarySemiCount = data.summarySemiCount.values.reshape(data.shape[0], 1)
summaryAtCount = data.summaryAtCount.values.reshape(data.shape[0], 1)
summaryDollaCount = data.summaryDollaCount.values.reshape(data.shape[0], 1)
summaryNumbCount = data.summaryNumbCount.values.reshape(data.shape[0], 1)
summaryAndCount = data.summaryAndCount.values.reshape(data.shape[0], 1)
summaryQuestionCount = data.summaryQuestionCount.values.reshape(data.shape[0], 1)
summaryAsterickCount = data.summaryAsterickCount.values.reshape(data.shape[0], 1)
summaryFParenthesisCount = data.summaryFParenthesisCount.values.reshape(data.shape[0], 1)
summaryBParenthesisCount = data.summaryBParenthesisCount.values.reshape(data.shape[0], 1)
# summaryPunctuationCount = data.summaryPunctuationCount.values.reshape(data.shape[0], 1)
# summaryPunctuationToWords = data.iloc[:, 70].values.reshape(data.shape[0], 1)
# summaryRepeatTxt = data.iloc[:, 71].values.reshape(data.shape[0], 1)


In [45]:

Xtoadd = np.concatenate((
#                          XScore, 
                         XhasEP, 
                         XhasSemi, 
                         XhasColon, 
                         hasAt, 
                         hasDolla, 
                         hasNumb, 
                         hasAnd, 
                         hasQuestion, 
                         hasAsterick, 
                         hasFparenthesis, 
                         hasBparenthesis, 
                         epCount, 
                         semiCount, 
                         atCount, 
                         dollaCount, 
                         numbCount, 
                         andCount, 
                         questionCount, 
                         asterickCount, 
                         fParenthesisCount, 
                         bParenthesisCount, 
#                          punctuationCount, 
#                          punctuationToWords, 
                         repeatTxt, 
                         nameNumb, 
                         nameQuestion, 
                         nameAsterick, 
                         nameAnd, 
                         nameSemi, 
                         nameColon, 
                         nameAt, 
                         nameDolla, 
#                          repeatProfile, 
#                          summaryLen, 
                         summaryEP, 
                         summarySemi, 
                         summaryColon, 
                         summaryAt, 
                         summaryDolla, 
                         summaryNumb, 
                         summaryAnd, 
                         summaryQuestion, 
                         summaryAsterick, 
                         summaryFparenthesis, 
                         summaryBparenthesis, 
                         summaryEpCount, 
                         summarySemiCount, 
                         summaryAtCount, 
                         summaryDollaCount, 
                         summaryNumbCount, 
                         summaryAndCount, 
                         summaryQuestionCount, 
                         summaryAsterickCount, 
                         summaryFParenthesisCount, 
                         summaryBParenthesisCount 
#                          summaryPunctuationCount, 
#                          summaryPunctuationToWords, 
#                          summaryRepeatTxt
    ), axis=1)

data.head()

,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,...,summarySemiCount,summaryAtCount,summaryDollaCount,summaryNumbCount,summaryAndCount,summaryQuestionCount,summaryAsterickCount,summaryFParenthesisCount,summaryBParenthesisCount,summaryRepeatTxt
0,138806,138807,B000E63LME,A1CQGW1AOD0LF2,"Alena K. ""Alena""",1,2,2,1294185600,Not as pictured.,...,0,0,1,0,0,0,0,0,0,False
1,469680,469681,B004ZIH4KM,A37S7U1OX2MCWI,Becky Cole,0,0,5,1349740800,seeds,...,0,0,1,0,0,0,0,0,0,False
2,238202,238203,B003ZXE9QA,A2OM6G73E64EQ9,jeff,0,0,5,1329264000,I'm addicted!,...,0,0,1,0,0,0,0,0,0,False
3,485307,485308,B001RVFERK,A25W349EE97NBK,Tangent4,1,1,4,1248307200,I wanted to love these...,...,0,0,1,0,0,0,0,0,0,False
4,375283,375284,B000OQZNTS,A3CPPW0HUC07YS,Amy Nicolai,0,0,5,1333238400,Excellent chamomile tea,...,0,0,1,0,0,0,0,0,0,False


In [46]:
Xfeatures = np.concatenate(( X_counts, Xtoadd), axis=1)


In [47]:
# y = data.iloc[:, 12].values
# y = np.where(y == 'true', -1, 1)

# X = data.iloc[:, 35].values #punctuation count

# plt.scatter(X[:50, 0], X[:50, 1],
# ... color='red', marker='o', label='helpfulness')

In [48]:
# report on training and test sets
global SVMerror, SVMacc, SVMtp, SVMtn, LRerror, LRacc, LRtp, LRtn, NBerror, NBacc, NBtp, NBtn, Perror, Pacc, Ptp, Ptn



def print_results(model):
    #print('Error rate on training set: ')
    erTRAIN = ((y_train != y_pred).sum() / X_train.shape[0])
    #print('Accuracy rate on training set: ')
    AccTRAIN = (1 - (y_train != y_pred).sum() / X_train.shape[0])
    #print('True positive rate on training tet:')
    TruPosTRAIN = ((y_train==True) & (y_pred==True)).sum() / y_train.sum()
    #TruNegTEST = (((y_train==False) & (y_pred_train==False)).sum() / (y_train.shape[0] - y_train.sum()))
    #print('**************')
    #('Error rate on test set: ')
    erTEST = ((y_test != y_pred_test).sum() / X_test.shape[0])
    #print('Accuracy rate on test set: ')
    AccTEST = (1 - (y_test != y_pred_test).sum() / X_test.shape[0])
    #print('True positive rate on test set')
    TruPosTEST = (((y_test==True) & (y_pred_test==True)).sum() / y_test.sum())
    #print('True negative rate on test set')
    TruNegTEST = (((y_test==False) & (y_pred_test==False)).sum() / (y_test.shape[0] - y_test.sum()))
    data_rows = [('Error Rate', erTRAIN, erTEST),
                 ('Accuracy Rate', AccTRAIN, AccTEST),
                 ('True Positives', TruPosTRAIN, TruPosTEST),
                 ('True Negatives', '--', TruNegTEST)]
    t = Table(rows=data_rows, names=(model, 'Training Set', 'Test Set'), meta={'name': model + ': Training and Test Set Results'})
    print(t)
    if model == 'SVM':
        SVMerror = erTEST
        SVMacc = AccTEST
        SVMtp = TruPosTEST
        SVMtn = TruNegTEST
        return(SVMerror, SVMacc, SVMtp, SVMtn)
    elif model == 'Logistic Regression':
        LRerror = erTEST
        LRacc = AccTEST
        LRtp = TruPosTEST
        LRtn = TruNegTEST
        return(LRerror, LRacc, LRtp, LRtn)
    elif model == 'Naive Bayes':
        NBerror = erTEST
        NBacc = AccTEST
        NBtp = TruPosTEST
        NBtn = TruNegTEST
        return(NBerror, NBacc, NBtp, NBtn)
    elif model == 'Perceptron':
        Perror = erTEST
        Pacc = AccTEST
        Ptp = TruPosTEST
        Ptn = TruNegTEST
        return(Perror, Pacc, Ptp, Ptn)
    print('done')
      
    #t.show_in_browser(jsviewer=True) 
    
    
def all_models_table():
    all_rows = [('SVM', SVMerror, SVMacc, SVMtp, SVMtn),
            ('Logistic Regression', LRerror, LRacc, LRtp, LRtn),
            ('Naive Bayes', NBerror, NBacc, NBtp, NBtn),
            ('Perceptron', Perror, Pacc, Ptp, Ptn)]
    tt = Table(rows=all_rows, names=('', 'Error Rate', 'Accuracy', 'True +', 'True -'), meta={'3/15/2016'})
    print(tt)

In [49]:
# convert to CSR
from scipy.sparse import csr_matrix, hstack
XtoaddSparse = csr_matrix(Xtoadd)
Xfinal = hstack([Xtoadd, XtoaddSparse,XhashText,XhashSummary,XText,XSummary])
X = csr_matrix(Xfinal)
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
X = imp.fit_transform(X)

TypeError: no supported conversion for types: (dtype('O'),)

In [50]:
# define y
# y = data.iloc[:, 12].values
# y.shape
y = data.helpful.values
y.shape



(455000,)

In [51]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [52]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [53]:
# decision tree classifier

# from sklearn import tree
# X = [[0, 0], [1, 1]]
# Y = [0, 1]
# clf = tree.DecisionTreeClassifier()
# clf = clf.fit(X, Y)

In [54]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM          Training Set      Test Set   
-------------- ----------------- --------------
    Error Rate 0.000863422291994 0.152827838828
 Accuracy Rate    0.999136577708 0.847172161172
True Positives    0.993972791459 0.446087738583
True Negatives                -- 0.878902389856


(0.15282783882783882,
 0.84717216117216121,
 0.44608773858299189,
 0.87890238985556512)

In [55]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- ---------------- --------------
         Error Rate 0.00110518053375 0.145465201465
      Accuracy Rate   0.998894819466 0.854534798535
     True Positives   0.994962975719 0.438892775057
     True Negatives               -- 0.887416694995


(0.14546520146520148,
 0.85453479853479852,
 0.43889277505745977,
 0.88741669499497999)

In [56]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set   
-------------- ---------------- ---------------
    Error Rate 0.00194662480377 0.0805054945055
 Accuracy Rate   0.998053375196  0.919494505495
True Positives   0.999354227656  0.328969721195
True Negatives               --   0.96621156902


(0.080505494505494504,
 0.91949450549450551,
 0.32896972119516338,
 0.96621156901962957)

In [57]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron      Training Set      Test Set   
-------------- ----------------- --------------
    Error Rate 0.000875981161695 0.152564102564
 Accuracy Rate    0.999124018838 0.847435897436
True Positives    0.994015842948 0.445188368142
True Negatives                -- 0.879258140767


(0.15256410256410258,
 0.84743589743589742,
 0.44518836814230039,
 0.87925814076668274)